# Grain Process

In [1]:
import yt.units as u
import numpy as np
atoms = ['H', 'C', 'O']
mass = {'H': 1.0079, 'C': 12.0107, 'O': 15.9994}

## Grain model

In [12]:
r = 1000 * u.angstrom
V = 4 / 3 * np.pi * r**3
rho = 3 * u.gram / u.cm**3
Ns = 1e6
G_M_ratio = 100
nH = 2e4 / u.cm**3
T = 10 * u.Kelvin
ns = nH * u.mass_hydrogen / G_M_ratio / rho / V

## Accretion

$$R_{\mathrm{acc}}(i)=\overline{\sigma_{d}n_{d}}\langle v(i)\rangle n(i) = \sqrt{\frac{2 k_{B} T}{\pi m}}\sigma_{d}n_{d}n(i)$$

$$k_i= \sqrt{\frac{2 k_{B} T}{\pi m}}\sigma_{d}n_{d}n(i)=6.0557\sqrt{\frac{T\ (\text{K})}{\mu}}\times10^{-14}\text{ s}^{-1}$$

In [32]:
def mu(spe):
    t = ''
    m = 0
    for i in spe:
        if i.isdigit():
            m += mass[t] * (int(i) - 1)
        else:
            t = i
            m += mass[t]
    return(m)

def acc(spe, label=0):
    t = ''
    m = 0
    grain_spe = ''
    for i in spe:
        if i.isdigit():
            grain_spe += (t + '_dust') * (int(i) - 1)
            m += mass[t] * (int(i) - 1)
        else:
            t = i
            grain_spe += (t + '_dust')
            m += mass[t]
    react = '{},{},{},NONE,NONE,6.0557e-14*sqrt(Tgas/{:f})'.format(
        label, spe, grain_spe, m)
    return(react)

In [33]:
LIST_complex = ['H','O','OH','H2O','C','CH','CH2','CH3','CH3OH','CO','HCO']
for i, spe in enumerate(LIST_complex):
    print(acc(spe, label=i))

0,H,H_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/1.007900)
1,O,O_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/15.999400)
2,OH,O_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/17.007300)
3,H2O,H_dustH_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/18.015200)
4,C,C_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/12.010700)
5,CH,C_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/13.018600)
6,CH2,C_dustH_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/14.026500)
7,CH3,C_dustH_dustH_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/15.034400)
8,CH3OH,C_dustH_dustH_dustH_dustO_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/32.041700)
9,CO,C_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/28.010100)
10,HCO,H_dustC_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/29.018000)


## Evaporation

### Thermal Evaporation

$$k_{\mathrm{eva, thermal}}(i)=\nu_{i} \exp \left[-E_{D} / kT\right]$$

$$\begin{aligned} \nu_{i} &=\left(2 \rho_{s} E_{D} / \pi^{2} m_{i}\right)^{1 / 2} \\ &=2.4 \times 10^{12} \mathrm{Hz}\left(\frac{\rho_{s}}{10^{15} \mathrm{cm}^{-2}}\right)^{1 / 2}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2} \end{aligned}$$

In [25]:
rhos = Ns/4/np.pi/r**2
np.sqrt(rhos.in_cgs()/1e15*u.cm**2)

0.8920620580763855 dimensionless

As a result,

$$k_{\mathrm{eva, thermal}}(i)=2.14 \times 10^{12} \mathrm{Hz}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2} \exp \left[-E_{D} / T\right]$$

### Cosmic Ray Desorption

$$k_{\mathrm{eva}, \mathrm{cosmic}}(i)=\zeta(70 \mathrm{K}) \cdot \nu_{i} \exp \left[-E_{D}/(70 \mathrm{K})\right]$$

$\zeta(70\text{K})\sim3.16 \times 10^{-19}$ (Hasegawa & Herbst, 1993), thus

$$k_{\mathrm{eva}, \mathrm{cosmic}}(i)=6.76\times10^{-7}\text{Hz}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2}\exp\left[-E_{D} / (70\text{K})\right]$$

In [37]:
def eva(spe, Eads, label=0):
    kth = 2.14e12/u.s*np.sqrt(Eads/350/u.Kelvin)/np.sqrt(mu(spe)/mu('H'))
    kcos = 6.76e-7/u.s*np.sqrt(Eads/350/u.Kelvin)/np.sqrt(mu(spe)/mu('H'))*np.exp(-Eads/70/u.Kelvin)
    t = ''
    grain_spe = ''
    for i in spe:
        if i.isdigit():
            grain_spe += (t + '_dust') * (int(i) - 1)
        else:
            t = i
            grain_spe += (t + '_dust')
    react = '{},{},{},NONE,NONE,{:e}*exp(-{:f}/Tgas)+{:e}'.format(
        label, grain_spe, spe, kth, Eads, kcos)
    return(react)

In [39]:
calorie = 41858000*u.erg
kcal = 1e3*calorie
Eads = {}
Eads['H'] = 350*u.Kelvin
Eads['H2'] = 450*u.Kelvin
Eads['C'] = 800*u.Kelvin
Eads['O'] = 800*u.Kelvin
Eads['CH'] = 1.3*kcal/6.02e23/u.boltzmann_constant
Eads['OH'] = 2.5*kcal/6.02e23/u.boltzmann_constant
Eads['CO'] = 2.4*kcal/6.02e23/u.boltzmann_constant
Eads['CH2'] = 1.9*kcal/6.02e23/u.boltzmann_constant
Eads['H2O'] = 2.9*kcal/6.02e23/u.boltzmann_constant
Eads['CH3'] = 2.3*kcal/6.02e23/u.boltzmann_constant
Eads['CH3OH'] = 4.1*kcal/6.02e23/u.boltzmann_constant
for item, i in enumerate(Eads.keys()):
    print(i)
    print(Eads[i])
    kth = 2.14e12/u.s*np.sqrt(Eads[i]/350/u.Kelvin)/np.sqrt(mu(i)/mu('H'))*np.exp(-Eads[i]/10/u.Kelvin)
    kcos = 6.76e-7/u.s*np.sqrt(Eads[i]/350/u.Kelvin)/np.sqrt(mu(i)/mu('H'))*np.exp(-Eads[i]/70/u.Kelvin)
    print('{:e} {:e}'.format(kth, kcos))
    print(eva(i, Eads[i], label=item))

H
350.0 K
1.349295e-03 4.554852e-09
0,H_dust,H,NONE,NONE,2.140000e+12*exp(-350.000000/Tgas)+4.554852e-09
H2
450.0 K
4.911559e-08 8.752071e-10
1,H_dustH_dust,H2,NONE,NONE,1.715817e+12*exp(-450.000000/Tgas)+8.752071e-10
C
800.0 K
1.691573e-23 3.221193e-12
2,C_dust,C,NONE,NONE,9.372367e+11*exp(-800.000000/Tgas)+3.221193e-12
O
800.0 K
1.465626e-23 2.790931e-12
3,O_dust,O,NONE,NONE,8.120478e+11*exp(-800.000000/Tgas)+2.790931e-12
CH
654.6996593219958 K
3.003188e-17 2.230839e-11
4,C_dustH_dust,CH,NONE,NONE,8.143805e+11*exp(-654.699659/Tgas)+2.230839e-11
OH
1259.0378063884534 K
2.067631e-43 4.819390e-15
5,O_dustH_dust,OH,NONE,NONE,9.880756e+11*exp(-1259.037806/Tgas)+4.819390e-15
CO
1208.6762941329152 K
2.429079e-41 7.555119e-15
6,C_dustO_dust,CO,NONE,NONE,7.543730e+11*exp(-1208.676294/Tgas)+7.555119e-15
CH2
956.8687328552245 K
2.634868e-30 3.467060e-13
7,C_dustH_dustH_dust,CH2,NONE,NONE,9.485058e+11*exp(-956.868733/Tgas)+3.467060e-13
H2O
1460.4838554106059 K
3.859307e-52 2.837306e-16
8,H_dustH